# Assign positives and negatives 

In [1]:
import itertools
import random
import sys
import json

import py2neo
import pandas

## Startup neo4j and connections

In [2]:
with open('servers.json') as read_file:
    instances = json.load(read_file)

for instance in instances:
    neo = py2neo.Graph(
        host="localhost", bolt_port=instance['port'],
        http_port=instance['port']+1, bolt=True
    )
    
    instance['py2neo'] = neo
    
    if instance['name'] == 'rephetio-v2.0':
        neo_unperm = instance['py2neo']

hetnets = [x['name'] for x in instances]
hetnets

['rephetio-v2.0',
 'rephetio-v2.0_perm-1',
 'rephetio-v2.0_perm-2',
 'rephetio-v2.0_perm-3',
 'rephetio-v2.0_perm-4',
 'rephetio-v2.0_perm-5']

## Create partitions

In [3]:
def to_df(record_list):
    """Convert a py2neo RecordList to a dataframe"""
    return pandas.DataFrame(record_list.data())

In [4]:
compound_df = pandas.read_table('../summary/compounds.tsv')
disease_df = pandas.read_table('../summary/diseases.tsv')

## All potential pairs

In [5]:
'{} compounds × {} diseases = {} pairs'.format(
    len(disease_df), len(compound_df), len(disease_df) * len(compound_df))

'134 compounds × 1393 diseases = 186662 pairs'

## Nonzero prior pairs

In [6]:
compound_df.query("treats > 0")["chemical_id"].nunique()

323

In [7]:
disease_df.query("treats > 0")["disease_id"].nunique()

73

In [8]:
nonzero_prior_pairs = set(itertools.product(
    compound_df.query("treats > 0").chemical_id,
    disease_df.query("treats > 0").disease_id)
)

In [9]:
len(nonzero_prior_pairs)

23579

---

## Read training data for cross validation

In [10]:
with open("../../crossval_idx.txt", "r") as fin:
    train_idx = int(fin.read().strip())

In [11]:
train_idx

0

In [12]:
train = pandas.read_csv("../../../crossval/data/training/training_piece{}.tsv".format(train_idx), sep = '\t')

In [13]:
train.shape

(38982, 6)

In [14]:
def df_to_pairs(df):
    return set(zip(df["chemical_id"], df["disease_id"]))    

In [15]:
train_pairs = df_to_pairs(train)

In [16]:
nonzero_prior_pairs <= train_pairs

False

There are some potential pairs which are present in the holdout set, so we will remove these from the potential negative training examples.

In [17]:
nonzero_prior_pairs &= train_pairs

In [18]:
len(nonzero_prior_pairs)

18403

In [19]:
nonzero_prior_pairs <= train_pairs

True

The data has now been sanitized so that no relations which appear in the holdout set are ever seen prior to making predictions.

---

## Split the data

In [20]:
indication_query = '''
MATCH (compound:Compound)-[treatment:TREATS_CtD]->(disease:Disease)
RETURN
  compound.identifier AS chemical_id,
  disease.identifier AS disease_id
ORDER BY
  chemical_id, disease_id
'''

def partition(neo):
    """
    Extract negative and positive compound-disease pairs from a py2neo.Graph.
    """
    
    # query hetnet for all positive drug-treat-disease examples
    # query hetnet for all potential drug-disease pairs
    # randomly sample 4x the number of non-positive examples for use as negative examples

    
    # Use TREATS_CtD as positives
    # different from training data because edges were permuted
    indication_df = to_df(neo.run(indication_query))
    
    positives = set(zip(indication_df.chemical_id, indication_df.disease_id))

    # Use nonzero-prior pairs excluding non-negatives as negatives
    negatives = nonzero_prior_pairs - positives
    
    assert negatives <= train_pairs, "Bad training data"
    
    
    negatives = random.sample(negatives, k=len(positives) * 4)
    rows = list()
    for status, pairs in (0, negatives), (1, positives):
        for drug, disease in pairs:
            rows.append((drug, disease, status))
    df = pandas.DataFrame(rows, columns=['chemical_id', 'disease_id', 'status'])
    df = df.sort_values(['disease_id', 'status', 'chemical_id'])
    return df

In [21]:
part_dfs = list()
for seed, instance in enumerate(instances):
    random.seed(seed, version=2)
    part_df = partition(instance['py2neo'])
    part_df.insert(0, 'hetnet', instance['name'])
    part_dfs.append(part_df)
part_df = pandas.concat(part_dfs)

In [22]:
part_df.head()

,hetnet,chemical_id,disease_id,status
1422,rephetio-v2.0,DB00153,DOID:0050741,0
1563,rephetio-v2.0,DB00178,DOID:0050741,0
689,rephetio-v2.0,DB00205,DOID:0050741,0
1333,rephetio-v2.0,DB00206,DOID:0050741,0
1760,rephetio-v2.0,DB00220,DOID:0050741,0


In [23]:
part_df.groupby("hetnet")["status"].value_counts()

hetnet                status
rephetio-v2.0         0         2368
                      1          592
rephetio-v2.0_perm-1  0         2368
                      1          592
rephetio-v2.0_perm-2  0         2368
                      1          592
rephetio-v2.0_perm-3  0         2368
                      1          592
rephetio-v2.0_perm-4  0         2368
                      1          592
rephetio-v2.0_perm-5  0         2368
                      1          592
Name: status, dtype: int64

In [24]:
## Create an even-matrix for unpermuted observations
template_df = pandas.DataFrame(list(itertools.product(hetnets, compound_df.chemical_id)), columns=['hetnet', 'chemical_id'])
unperm_pair_df = part_df.query("hetnet == 'rephetio-v2.0'")[['chemical_id', 'disease_id']]
unperm_pair_df = template_df.merge(unperm_pair_df)
unperm_pair_df = unperm_pair_df.merge(part_df, how='left')
unperm_pair_df = unperm_pair_df[unperm_pair_df.status.isnull()]
unperm_pair_df.status = unperm_pair_df.status.fillna(0).astype(int)
unperm_pair_df.head(2)

,hetnet,chemical_id,disease_id,status
9,rephetio-v2.0_perm-1,DB01048,DOID:10652,0
10,rephetio-v2.0_perm-1,DB01048,DOID:12361,0


In [25]:
part_df['primary'] = 1
unperm_pair_df['primary'] = 0
full_part_df = pandas.concat([part_df, unperm_pair_df])
full_part_df = full_part_df.sort_values(['chemical_id', 'disease_id', 'hetnet'])
full_part_df.head(5)

,hetnet,chemical_id,disease_id,status,primary
1865,rephetio-v2.0_perm-1,DB00014,DOID:0050742,0,1
2375,rephetio-v2.0_perm-2,DB00014,DOID:0050742,1,1
694,rephetio-v2.0_perm-3,DB00014,DOID:0060073,0,1
345,rephetio-v2.0_perm-5,DB00014,DOID:0060073,0,1
2946,rephetio-v2.0,DB00014,DOID:10283,1,1


In [26]:
full_part_df.to_csv('data/partitions.tsv', sep='\t', index=False)

In [27]:
# Number of hetnet-compound-disease pairs
len(full_part_df)

30203

In [28]:
# Number of positives and negatives per purpose
pandas.crosstab(full_part_df.primary, full_part_df.status)

status,0,1
primary,,
0,12443,0
1,14208,3552


In [29]:
# Number of positives and negatives per hetnet
pandas.crosstab(full_part_df.hetnet, full_part_df.status)

status,0,1
hetnet,,
rephetio-v2.0,2368,592
rephetio-v2.0_perm-1,4870,592
rephetio-v2.0_perm-2,4847,592
rephetio-v2.0_perm-3,4856,592
rephetio-v2.0_perm-4,4847,592
rephetio-v2.0_perm-5,4863,592
